In [1]:
import torch
from torch.nn import functional as F

from fse.models.base_s2v import BaseSentence2VecModel
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

import pandas as pd
import swifter
from tqdm.notebook import tqdm
import csv
import pickle
import time

/usr/local/lib/python3.7/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
data_path = '../../data_2020/'
model_path = 'BioSentVec_PubMed_MIMICIII-bigram_d700.bin'
other_path = 'others/'
prefix = 'test'
postpostfix = '_'+prefix
MIN_LEN = 7
t1 = time.time()
paper_thd = 45000

In [3]:
model = sent2vec.Sent2vecModel()
model.load_model(model_path)
data = pd.read_csv(data_path+prefix+'_pre.csv')[['description_id', 'description_text', 'key_text']].fillna('')
paper = pd.read_csv(data_path+'candidate_paper_pre.csv')[['title', 'paper_id']].fillna('')
dids = data['description_id'].values

pid2idx = {}
idx2pid = {}
for i, pid in enumerate(tqdm(paper['paper_id'].values)):
    pid2idx[pid] = len(pid2idx)
    idx2pid[len(pid2idx)-1] = pid
assert len(pid2idx) == paper.shape[0]

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()
    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]
    return ' '.join(tokens)

data['key_text_len'] = data['key_text'].apply(lambda x: len(x.split()))
data.loc[data['key_text_len'] < MIN_LEN, 'key_text'] = data[data['key_text_len'] < MIN_LEN]['description_text'].values
data = data['key_text'].swifter.allow_dask_on_strings().apply(lambda x: preprocess_sentence(x)).values
paper = paper['title'].swifter.allow_dask_on_strings().apply(lambda x: preprocess_sentence(x)).values

data[0], paper[0]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


('refer table 2 methods brief description variables additionally vut dummy vanuatu fri kri mon dummies three studies incomplete cost categories ** ** colons indicated interaction terms',
 'anopheles stephensi effect gamma-radiation chemosterilants fertility fitness males sterile male releases')

In [4]:
data_embs = [model.embed_sentence(sentence)[0] for sentence in tqdm(data)]
paper_embs = [model.embed_sentence(sentence)[0] for sentence in tqdm(paper)]

print('generating tensors...')
data_embs = torch.FloatTensor(data_embs)
paper_embs = torch.FloatTensor(paper_embs)
data_embs = F.normalize(data_embs, p=2, dim=1)
paper_embs = F.normalize(paper_embs, p=2, dim=1)

batch_size = 2048
steps = data_embs.size(0)//batch_size + 1
topis = []

for step in tqdm(range(steps)):
    data_embs_b = data_embs[step*batch_size: (step+1)*batch_size]
    results = torch.mm(data_embs_b, paper_embs.transpose(0, 1))
    _, topi = torch.topk(results, 1000)
    topis += topi.tolist()

with open(data_path+prefix+'_pairs_1000number_s2v_2.csv', 'w', newline='') as f:
    w = csv.writer(f)
    for i in range(data.shape[0]):
        w.writerow([dids[i]]+[idx2pid[idx] for idx in topis[i]])
        
t2 = time.time()
t2-t1


generating tensors...


264.91659140586853

### recall score for train
***

In [5]:
data = pd.read_csv(data_path+prefix+'_pre.csv')[['description_id', 'paper_id']].values
hit = 0

for i in tqdm(range(data.shape[0])):
    for j in range(3):
        if topis[i][j] == pid2idx[data[i][1]]:
            hit += 1
            break
            
hit/data.shape[0]

KeyError: "['paper_id'] not in index"